[View in Colaboratory](https://colab.research.google.com/github/Curiousss/InkerIntern/blob/master/Copy_of_DNST_CIFAR10_AUG.ipynb)

In [0]:
# https://keras.io/
!pip install -q keras
import keras

In [0]:
import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam

In [0]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [0]:
# Hyperparameters
batch_size = 128
num_classes = 10
epochs = 50
l = 40
num_filter = 12
compression = 0.5
dropout_rate = 0.2

In [0]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [0]:
# Dense Block
def add_denseblock(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

In [0]:
def add_transition(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg

In [0]:
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output

In [0]:
num_filter = 12
dropout_rate = 0.2
l = 12
input = Input(shape=(img_height, img_width, channel,))
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)


In [24]:
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_53 (Conv2D)              (None, 32, 32, 12)   324         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_53 (BatchNo (None, 32, 32, 12)   48          conv2d_53[0][0]                  
__________________________________________________________________________________________________
activation_53 (Activation)      (None, 32, 32, 12)   0           batch_normalization_53[0][0]     
__________________________________________________________________________________________________
conv2d_54 

In [0]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

In [26]:
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/50
50000/50000 [==============================] - 179s 4ms/step - loss: 1.6742 - acc: 0.3695 - val_loss: 1.6818 - val_acc: 0.4094
Epoch 2/50
19200/50000 [==========>...................] - ETA: 1:37 - loss: 1.3841 - acc: 0.4880

50000/50000 [==============================] - 167s 3ms/step - loss: 1.3343 - acc: 0.5076 - val_loss: 1.3983 - val_acc: 0.5043
Epoch 3/50
45568/50000 [==========================>...] - ETA: 14s - loss: 1.1976 - acc: 0.5614

50000/50000 [==============================] - 167s 3ms/step - loss: 1.1936 - acc: 0.5636 - val_loss: 1.6512 - val_acc: 0.4970
Epoch 4/50
50000/50000 [==============================] - 166s 3ms/step - loss: 1.0958 - acc: 0.6012 - val_loss: 1.2648 - val_acc: 0.5953
Epoch 5/50
 3328/50000 [>.............................] - ETA: 2:27 - loss: 1.0384 - acc: 0.6160

50000/50000 [==============================] - 166s 3ms/step - loss: 1.0286 - acc: 0.6279 - val_loss: 1.6515 - val_acc: 0.5284
Epoch 6/50
39552/50000 [======================>.......] - ETA: 32s - loss: 0.9821 - acc: 0.6472

50000/50000 [==============================] - 166s 3ms/step - loss: 0.9781 - acc: 0.6486 - val_loss: 1.3870 - val_acc: 0.5758
Epoch 7/50
50000/50000 [==============================] - 167s 3ms/step - loss: 0.9385 - acc: 0.6620 - val_loss: 0.9783 - val_acc: 0.6616
Epoch 8/50
 1920/50000 [>.............................] - ETA: 2:32 - loss: 0.9197 - acc: 0.6760

...

50000/50000 [==============================] - 165s 3ms/step - loss: 0.5323 - acc: 0.8132 - val_loss: 0.8317 - val_acc: 0.7533
Epoch 45/50
38656/50000 [======================>.......] - ETA: 35s - loss: 0.5274 - acc: 0.8133

50000/50000 [==============================] - 165s 3ms/step - loss: 0.5255 - acc: 0.8142 - val_loss: 0.8403 - val_acc: 0.7505
Epoch 46/50
50000/50000 [==============================] - 165s 3ms/step - loss: 0.5227 - acc: 0.8148 - val_loss: 1.4730 - val_acc: 0.6517
Epoch 47/50
 1536/50000 [..............................] - ETA: 2:30 - loss: 0.4657 - acc: 0.8288

50000/50000 [==============================] - 165s 3ms/step - loss: 0.5233 - acc: 0.8161 - val_loss: 0.7863 - val_acc: 0.7631
Epoch 48/50
38656/50000 [======================>.......] - ETA: 35s - loss: 0.5200 - acc: 0.8157

50000/50000 [==============================] - 165s 3ms/step - loss: 0.5172 - acc: 0.8164 - val_loss: 1.1183 - val_acc: 0.6999
Epoch 49/50
50000/50000 [==============================] - 165s 3ms/step - loss: 0.5161 - acc: 0.8188 - val_loss: 0.7066 - val_acc: 0.7847
Epoch 50/50
 1536/50000 [..............................] - ETA: 2:31 - loss: 0.5318 - acc: 0.8105

50000/50000 [==============================] - 165s 3ms/step - loss: 0.5088 - acc: 0.8212 - val_loss: 0.9576 - val_acc: 0.7293


In [27]:
# Test the model
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 13s 1ms/step
Test loss: 0.9576269150733948
Test accuracy: 0.7293


In [28]:
# Save the trained weights in to .h5 format
model.save_weights("DNST_model.h5")
print("Saved model to disk")

Saved model to disk


In [0]:
from google.colab import files

files.download('DNST_model.h5')